##### ARTI 560 - Computer Vision  
## Image Classification using Transfer Learning - Exercise 

### Objective

In this exercise, you will:

1. Select another pretrained model (e.g., VGG16, MobileNetV2, or EfficientNet) and fine-tune it for CIFAR-10 classification.  
You'll find the pretrained models in [Tensorflow Keras Applications Module](https://www.tensorflow.org/api_docs/python/tf/keras/applications).

2. Before training, inspect the architecture using model.summary() and observe:
- Network depth
- Number of parameters
- Trainable vs Frozen layers

3. Then compare its performance with ResNet and the custom CNN.

### Questions:

- Which model achieved the highest accuracy?
- Which model trained faster?
- How might the architecture explain the differences?

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

# -----------------------------
# 1) Load CIFAR-10
# -----------------------------
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

class_names = [
    "airplane","automobile","bird","cat","deer",
    "dog","frog","horse","ship","truck"
]

# Keep labels as integers
y_train = y_train.squeeze().astype("int64")
y_test  = y_test.squeeze().astype("int64")

# Convert images to float32
x_train = x_train.astype("float32")
x_test  = x_test.astype("float32")

# -----------------------------
# 2) Data augmentation
# -----------------------------
data_augmentation = keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.05),
    layers.RandomZoom(0.1),
], name="augmentation")

# -----------------------------
# 3) Build MobileNetV2 backbone (pretrained)
# -----------------------------
mobilenet_base = MobileNetV2(
    include_top=False,
    weights="imagenet",
    input_shape=(224, 224, 3)
)

mobilenet_base.trainable = False   # freeze backbone

# -----------------------------
# 4) Full model (preprocess inside model)
# -----------------------------
mobilenet_model = keras.Sequential([
    layers.Input(shape=(32, 32, 3)),
    data_augmentation,
    layers.Resizing(224, 224, interpolation="bilinear"),
    layers.Lambda(preprocess_input),      # IMPORTANT for MobileNetV2
    mobilenet_base,
    layers.GlobalAveragePooling2D(),
    layers.Dense(10)                      # logits
], name="cifar10_mobilenetv2")

mobilenet_model.summary()

# -----------------------------
# 5) Compile + Train (frozen backbone)
# -----------------------------
mobilenet_model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"]
)

callbacks = [
    keras.callbacks.EarlyStopping(
        monitor="val_accuracy",
        patience=3,
        restore_best_weights=True
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss",
        factor=0.5,
        patience=1
    ),
]

history = mobilenet_model.fit(
    x_train, y_train,
    validation_split=0.1,
    epochs=3,
    batch_size=64,
    callbacks=callbacks,
    verbose=1
)



170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "cifar10_mobilenetv2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ augmentation (Sequential)       │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ resizing (Resizing)             │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lambda (Lambda)                 │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 10)             │        12,810 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,270,794 (8.66 MB)

 Trainable params: 12,810 (50.04 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

Epoch 1/3
704/704 ━━━━━━━━━━━━━━━━━━━━ 53s 65ms/step - accuracy: 0.5983 - loss: 1.1552 - val_accuracy: 0.8020 - val_loss: 0.5644 - learning_rate: 0.0010
Epoch 2/3
704/704 ━━━━━━━━━━━━━━━━━━━━ 44s 63ms/step - accuracy: 0.7463 - loss: 0.7292 - val_accuracy: 0.8220 - val_loss: 0.5243 - learning_rate: 0.0010
Epoch 3/3
704/704 ━━━━━━━━━━━━━━━━━━━━ 44s 63ms/step - accuracy: 0.7599 - loss: 0.6936 - val_accuracy: 0.8266 - val_loss: 0.4985 - learning_rate: 0.0010


In [2]:
test_loss, test_acc_m = mobilenet_model.evaluate(x_test, y_test, verbose=0)

print("MobileNetV2 (frozen) test accuracy:", test_acc_m)
print("MobileNetV2 (frozen) test loss    :", test_loss)


MobileNetV2 (frozen) test accuracy: 0.8220999836921692
MobileNetV2 (frozen) test loss    : 0.5219805240631104


In [3]:
# -----------------------------
# Fine-tune last layers (MobileNetV2)
# -----------------------------
mobilenet_base.trainable = True

# Unfreeze last 30 layers
for layer in mobilenet_base.layers[:-30]:
    layer.trainable = False

print(
    "Trainable layers in backbone:",
    sum(l.trainable for l in mobilenet_base.layers),
    "/",
    len(mobilenet_base.layers)
)

mobilenet_model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-5),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"]
)

history_ft = mobilenet_model.fit(
    x_train, y_train,
    validation_split=0.1,
    epochs=3,
    batch_size=64,
    verbose=1
)

# -----------------------------
# Evaluate Fine-tuned Model
# -----------------------------
test_loss_ft, test_acc_ft = mobilenet_model.evaluate(x_test, y_test, verbose=0)

print("MobileNetV2 (fine-tuned) test accuracy:", test_acc_ft)
print("MobileNetV2 (fine-tuned) test loss    :", test_loss_ft)

Trainable layers in backbone: 30 / 154
Epoch 1/3
704/704 ━━━━━━━━━━━━━━━━━━━━ 66s 79ms/step - accuracy: 0.6796 - loss: 0.9338 - val_accuracy: 0.8296 - val_loss: 0.5016
Epoch 2/3
704/704 ━━━━━━━━━━━━━━━━━━━━ 54s 76ms/step - accuracy: 0.7702 - loss: 0.6542 - val_accuracy: 0.8386 - val_loss: 0.4672
Epoch 3/3
704/704 ━━━━━━━━━━━━━━━━━━━━ 54s 76ms/step - accuracy: 0.7918 - loss: 0.5925 - val_accuracy: 0.8488 - val_loss: 0.4238
MobileNetV2 (fine-tuned) test accuracy: 0.8510000109672546
MobileNetV2 (fine-tuned) test loss    : 0.44149574637413025


In [ ]:
#Custom CNN test acc: 0.8741999864578247

#ResNet frozen test acc: None

#ResNet fine-tuned test acc: 0.9161999821662903

#MobileNetV2 (fine-tuned) test accuracy: 0.8510000109672546

#MobileNetV2 (frozen) test accuracy: 0.8220999836921692

#"Mobilenet achived the lowest accuracy"

In [ ]:
#Total params: 2,270,794 (8.66 MB)
 #Trainable params: 12,810 (50.04 KB)
 #Non-trainable params: 2,257,984 (8.61 MB)

In [ ]:
#Which model achieved the highest accuracy?
#ResNet (fine-tuned) had the highest accuracy (about 91.6%).
#Because it was trained more deeply and learned better features.

#Which model trained faster?
#MobileNetV2 (frozen) trained the fastest.
#Because most of its layers were not trained, so it needed less time.

#How does the architecture explain the differences?
#Custom CNN: Simple and small, so it cannot learn very complex patterns.
#ResNet: Deep and powerful, so it learns better features and gives higher accuracy.
#MobileNetV2: Designed to be fast and lightweight, so it is faster but less accurate.